In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/diabetes.csv')

In [3]:
df.sample()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
733,2,106,56,27,165,29.0,0.426,22,0


In [4]:
import numpy as np

In [5]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
x = df.drop('Outcome', axis=1)
y = df['Outcome']

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x = scaler.fit_transform(x)

In [8]:
x.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [10]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
model = Sequential()

model.add(Dense(32, activation='relu', input_shape=(8,)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\jhota\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.fit(x_train, y_train, epochs=100, batch_size=1, validation_data=(x_test, y_test))

Epoch 1/100
614/614 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6130 - loss: 0.6524 - val_accuracy: 0.7403 - val_loss: 0.5498
Epoch 2/100
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7796 - loss: 0.4842 - val_accuracy: 0.7922 - val_loss: 0.5247
Epoch 3/100
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7667 - loss: 0.4623 - val_accuracy: 0.7727 - val_loss: 0.5168
Epoch 4/100
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7611 - loss: 0.4661 - val_accuracy: 0.7857 - val_loss: 0.5180
Epoch 5/100
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7771 - loss: 0.4549 - val_accuracy: 0.7792 - val_loss: 0.5204
Epoch 6/100
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8013 - loss: 0.4171 - val_accuracy: 0.7597 - val_loss: 0.5247
Epoch 7/100
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7998 - loss: 0.4710 - val_accuracy: 0.7662 - val_loss: 0.5215
Epoch 8/100
614/614 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7868 - loss: 0.4417 - val_accu

In [14]:
## *How to select Apprioriate Optimizers*
### no of nodes in layer
### no of layers
### All in model

In [16]:
! pip install keras_tuner

  Using cached keras_tuner-1.4.7-py3-none-any.whl.metadata (5.4 kB)
  Using cached kt_legacy-1.0.5-py3-none-any.whl.metadata (221 bytes)
Using cached keras_tuner-1.4.7-py3-none-any.whl (129 kB)
Using cached kt_legacy-1.0.5-py3-none-any.whl (9.6 kB)


In [17]:
import keras_tuner as kt

In [18]:
def build_model(hp):

    model= Sequential()

    model.add(Dense(32, activation='relu', input_shape=(8,)))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [19]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

c:\Users\jhota\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test),verbose=1)

In [25]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [26]:
model = tuner.get_best_models()[0]

c:\Users\jhota\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\jhota\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test),initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7391 - loss: 0.5324 - val_accuracy: 0.7727 - val_loss: 0.5254
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7451 - loss: 0.5307 - val_accuracy: 0.7727 - val_loss: 0.5156
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7617 - loss: 0.4863 - val_accuracy: 0.7792 - val_loss: 0.5088
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7813 - loss: 0.4783 - val_accuracy: 0.7662 - val_loss: 0.5056
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7649 - loss: 0.4737 - val_accuracy: 0.7597 - val_loss: 0.5034
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7489 - loss: 0.4811 - val_accuracy: 0.7597 - val_loss: 0.5013
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7531 - loss: 0.4773 - val_accuracy: 0.7727 - val_loss: 0.4997
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7794 - loss: 0.4544 - val_accuracy: 0.772

In [27]:
# No of neurons

In [28]:
def build_model(hp):

    model = Sequential()

    units = hp.Int('units', min_value=8, max_value=128, step=8)

    model.add(Dense(units, activation='relu', input_shape=(8,)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [29]:
tuner = kt.RandomSearch(build_model, 
                        objective='val_accuracy',
                        max_trials=5,
                        directory='output',
                        project_name='my_project')

In [30]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6883116960525513

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 13s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'units': 88}

In [32]:
model = tuner.get_best_models(num_models=1)[0]

In [37]:
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test), initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7527 - loss: 0.5759 - val_accuracy: 0.7662 - val_loss: 0.5388
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7344 - loss: 0.5494 - val_accuracy: 0.7727 - val_loss: 0.5224
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7637 - loss: 0.4942 - val_accuracy: 0.7532 - val_loss: 0.5131
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7659 - loss: 0.4868 - val_accuracy: 0.7532 - val_loss: 0.5064
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7580 - loss: 0.4883 - val_accuracy: 0.7468 - val_loss: 0.5031
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7753 - loss: 0.4649 - val_accuracy: 0.7532 - val_loss: 0.5009
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7515 - loss: 0.4851 - val_accuracy: 0.7597 - val_loss: 0.4993
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7820 - loss: 0.4452 - val_accuracy: 0.766

In [38]:
# how to sleect no of layers

In [ ]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(88, activation='relu', input_shape=(8,)))

    for i in range(hp.Int('num_layers', 1, 10)):
        model.add(Dense(40, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [40]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='output-layer',
                        project_name='my_project')

c:\Users\jhota\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7337662577629089

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 16s


In [42]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5}

In [43]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\jhota\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\jhota\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [44]:
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test), initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7952 - loss: 0.4201 - val_accuracy: 0.7727 - val_loss: 0.5374
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7934 - loss: 0.4211 - val_accuracy: 0.7143 - val_loss: 0.5085
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8121 - loss: 0.4055 - val_accuracy: 0.7792 - val_loss: 0.5463
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7849 - loss: 0.4250 - val_accuracy: 0.7143 - val_loss: 0.5259
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8050 - loss: 0.3996 - val_accuracy: 0.6948 - val_loss: 0.5535
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7977 - loss: 0.4125 - val_accuracy: 0.7208 - val_loss: 0.5513
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8134 - loss: 0.3963 - val_accuracy: 0.7013 - val_loss: 0.5545
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8093 - loss: 0.4006 - val_accuracy: 0.71

In [46]:
def build_model(hp):

    model = Sequential()

    counter =0

    for i in range(hp.Int('num_layers', 1, 10)):
        if counter ==0:
            model.add(Dense(hp.Int('units_' + str(i), 8, 128, 8), 
                            activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh', 'sigmoid']),
                            input_shape=(8,)))
        else:
            model.add(Dense(hp.Int('units_' + str(i), 8, 128, 8), 
                            activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh', 'sigmoid'])))
        counter +=1

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimier', values=['rmsprop','adam','sgd','adadelta']), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [47]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='output-activation',
                        project_name='my_project')

c:\Users\jhota\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [48]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 13s


In [49]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units_0': 80,
 'activation_0': 'tanh',
 'optimier': 'rmsprop',
 'units_1': 8,
 'activation_1': 'sigmoid',
 'units_2': 104,
 'activation_2': 'tanh',
 'units_3': 88,
 'activation_3': 'relu',
 'units_4': 40,
 'activation_4': 'relu',
 'units_5': 80,
 'activation_5': 'sigmoid',
 'units_6': 120,
 'activation_6': 'sigmoid',
 'units_7': 112,
 'activation_7': 'relu',
 'units_8': 112,
 'activation_8': 'relu',
 'units_9': 88,
 'activation_9': 'sigmoid'}

In [50]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\jhota\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\jhota\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [52]:
model.fit(x_train, y_train, epochs=200, batch_size=32, validation_data=(x_test, y_test), initial_epoch=6)

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8204 - loss: 0.4131 - val_accuracy: 0.7403 - val_loss: 0.5185
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8408 - loss: 0.3720 - val_accuracy: 0.7208 - val_loss: 0.5617
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8150 - loss: 0.3847 - val_accuracy: 0.7273 - val_loss: 0.5332
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8012 - loss: 0.4011 - val_accuracy: 0.7468 - val_loss: 0.5210
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8138 - loss: 0.3987 - val_accuracy: 0.7208 - val_loss: 0.5541
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8361 - loss: 0.3811 - val_accuracy: 0.7468 - val_loss: 0.5197
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8051 - loss: 0.4145 - val_accuracy: 0.7468 - val_loss: 0.5209
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7967 - loss: 0.4283 - val_accuracy: 0.753